### GET Soil Adjusted Vegetation Index (SAVI)

Dataset Used: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_TOA#description

### IMPORTS

In [1]:
import time
from typing import Any

import ee
import geemap
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

### AUTHENTICATE AND INITIALIZE

In [2]:
# Authenticate to the Earth Engine servers
ee.Authenticate()

# Initialize the Earth Engine module.
ee.Initialize()

### VARIABLES

In [3]:
DATASET_NAME: str = "LANDSAT/LC09/C02/T1_TOA"
START_DATE: str = "2023-01-01"
END_DATE: str = "2024-01-01"
SCALE: int = 30
# ZONES: range = range(2, 10)
ZONES = [4, 9]
MAXPIXELS = 1e10

### FUNCTIONS

In [4]:
def cal_savi(image):
    savi = image.expression(
        '((NIR - RED) / (NIR + RED + L)) * (1 + L)',
        {
            'NIR': image.select('B5'),
            'RED': image.select('B4'),
            'L': 0.5  # Soil Brightness Adjustment Factor
        }
    ).rename('SAVI').copyProperties(image, ['system:time_start'])
    return savi

In [5]:
def clip_image(image):
    return image.clip(roi.geometry())

### DOWNLOAD IMAGES

In [6]:
for zone in tqdm(ZONES):
    shapefile_path = f"/vsizip/../shape_files/zone_{zone}.zip/layers/POLYGON.shp"
    gdf = gpd.read_file(shapefile_path)
    roi = geemap.gdf_to_ee(gdf)

    COLLECTION = ee.ImageCollection(DATASET_NAME)

    FILTERED = COLLECTION.filter(ee.Filter.date(START_DATE, END_DATE)).filter(ee.Filter.bounds(roi.geometry())) #.median()

    # Apply the expression
    SAVI_COLLECTION = FILTERED.map(cal_savi)
    
    # Clip
    SAVI_CLIPED = SAVI_COLLECTION.map(clip_image)
    
    # Median
    MEDIAN = SAVI_CLIPED.median()

    # EXPORTING THE MEDIAN IMAGE
    print(f"EXPORTING MEDIAN IMAGES FOR Zone {zone}")
    task_median = ee.batch.Export.image.toDrive(
        image=MEDIAN,  # Bands
        description=f"Median_Image_Zone{zone}",
        fileNamePrefix=f"zone{zone}_savi_median_{START_DATE}_{END_DATE}",
        folder=f"savi_roi_median_30m",
        scale=SCALE,
        region=roi.geometry().getInfo()["coordinates"],
        maxPixels=MAXPIXELS
    )
    
    task_median.start()
    while task_median.active():
        print("Polling for task (id: {}).".format(task_median.id))
        time.sleep(5)  # Poll every 5 seconds

    
    print(f"Task for Zone {zone} median image download completed with state {task_median.status()['state']}: ")

    # EXPORTING ALL IMAGES WITH 30m RESOLUTION (refer to dataset)
    print("#" * 50)
    print(f"EXPORTING ALL IMAGES FOR Zone {zone}")
    image_ids = SAVI_CLIPED.aggregate_array("system:index").getInfo()
    print("#" * 50)
    print("Total Images: ", len(image_ids))
    print("#" * 50)

    for i, image_id in enumerate(tqdm(image_ids, desc="Exporting Images")):
        image = ee.Image(SAVI_CLIPED.filter(ee.Filter.eq("system:index", image_id)).first())
        task = ee.batch.Export.image.toDrive(
        **{
            "image": image,
            "description": "Image Export {}".format(i + 1),
            "fileNamePrefix": f"zone{zone}_savi_{image.id().getInfo()}",
            "folder": f"savi_roi_30m",
            "scale": SCALE,
            "region": roi.geometry().getInfo()["coordinates"], # Use ROI geometry
            # "region": image.geometry().bounds().getInfo()["coordinates"],
            "maxPixels": MAXPIXELS,
        })
        task.start()
        # Monitor the task
        while task.active():
            print("Polling for task (id: {}).".format(task.id))
            time.sleep(5)  # Poll every 5 seconds
        
        print(f"Task {i + 1} completed with state {task.status()['state']}: ")    

  0%|          | 0/2 [00:00<?, ?it/s]

EXPORTING MEDIAN IMAGES FOR Zone 4
Polling for task (id: VT37TUXY2P3WXXW2XRRRXK6Y).
Polling for task (id: VT37TUXY2P3WXXW2XRRRXK6Y).
Polling for task (id: VT37TUXY2P3WXXW2XRRRXK6Y).
Polling for task (id: VT37TUXY2P3WXXW2XRRRXK6Y).
Task for Zone 4 median image download completed with state COMPLETED: 
EXPORTING ALL IMAGES FOR Zone 4
##################################################
Total Images:  55
##################################################



Exporting Images:   0%|          | 0/55 [00:00<?, ?it/s]

Polling for task (id: HFYQMRXGZF3HE3GABAAPX5JL).
Polling for task (id: HFYQMRXGZF3HE3GABAAPX5JL).
Polling for task (id: HFYQMRXGZF3HE3GABAAPX5JL).
Polling for task (id: HFYQMRXGZF3HE3GABAAPX5JL).



Exporting Images:   2%|▏         | 1/55 [00:23<21:08, 23.50s/it]

...

Task 53 completed with state COMPLETED: 
Polling for task (id: 2Y34M7VK6EN5KJS4VP6YL5F7).
Polling for task (id: 2Y34M7VK6EN5KJS4VP6YL5F7).
Polling for task (id: 2Y34M7VK6EN5KJS4VP6YL5F7).
Polling for task (id: 2Y34M7VK6EN5KJS4VP6YL5F7).



Exporting Images:  98%|█████████▊| 54/55 [20:16<00:22, 22.20s/it]

Task 54 completed with state COMPLETED: 
Polling for task (id: 4XLHAKWNBZVCDXJH3D2KLYQL).
Polling for task (id: 4XLHAKWNBZVCDXJH3D2KLYQL).
Polling for task (id: 4XLHAKWNBZVCDXJH3D2KLYQL).



100%|██████████| 2/2 [41:38<00:00, 1249.19s/it]

Task 55 completed with state COMPLETED: 
